In [127]:
import numpy as np
import scipy as sp
import pandas as pd

from sklearn.linear_model import LinearRegression

In [128]:
# reading data
data = pd.read_csv("year_prediction.csv")
data = data.rename(index=str, columns={"label":"year"})

In [129]:
# separate input attributes and output into different dataframes
X = data.iloc[:,1:]
Y = data.iloc[:,0]

# Train set
X_train = X.iloc[0:463715,:]
y_train = Y.iloc[0:463715]

# Validation set
X_test = X.iloc[463715:,:]
y_test = Y.iloc[463715:]

In [198]:
X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
linearReg = LinearRegression()
linearReg.fit(X_train, y_train)

LinearRegression()

In [199]:
pred = linearReg.predict(X_train)
total_cost = np.sum((pred - y_train) ** 2) / 463715
print(total_cost)

91.2564266561149


In [275]:
# construct coreset
def coreset_lr(m, D, y):
    # m is the number of samples, X is the data
    num_of_data, _ = D.shape
    D = np.hstack((D,y.reshape(-1,1)))
    #s = np.sum(D ** 2, axis=1) + 1 / num_of_data
    D1 = D[:,:30]
    D2 = D[:,30:60]
    D3 = D[:,60:]
    q1, _ = np.linalg.qr(D1)
    q2, _ = np.linalg.qr(D2)
    q3, _ = np.linalg.qr(D2)
    Q = np.hstack((q1,q2,q3))
    
    s = np.sum(Q ** 2, axis=1) + 1/num_of_data
    print(s[:10])
    #D = np.hstack((D, y.to_numpy().reshape(-1,1)))
    D = np.hstack((D, (1/s).reshape(-1,1)))
    D_df = pd.DataFrame(D)
    C = D_df.sample(n=m, replace=False, weights=s)
    C = C.to_numpy()
    data = C[:,:-2]
    label = C[:,-2]
    weight = C[:,-1]
    print(data.shape)
    return data, label, weight.reshape(-1)

def uniform_lr(m,D,y):
    D = np.hstack((D, y.reshape(-1,1)))
    #D = np.hstack((D, y.to_numpy().reshape(-1,1)))
    D_df = pd.DataFrame(D)
    C = D_df.sample(n=m, replace=False)
    C = C.to_numpy()
    data = C[:,:-1]
    label = C[:,-1]
    return data, label

In [284]:
coreset_X, coreset_y, weight = coreset_lr(5000,X_train,y_train)
linearReg.fit(coreset_X, coreset_y, weight)

#coreset_X, coreset_y, _ = coreset_lr(5000,X_train,y_train)
#linearReg.fit(coreset_X, coreset_y)

pred = linearReg.predict(X_train)
total_cost = np.sum((pred - y_train.reshape(-1)) ** 2) / 463715
print(total_cost)

[4.64233677e-05 8.44351724e-05 3.86045718e-05 6.43181809e-05
 4.97029785e-05 5.35264684e-05 1.61258768e-04 6.63347188e-05
 4.04543119e-05 2.00939575e-04]
(5000, 90)
93.1354558205793


In [285]:
uniform_X, uniform_y = uniform_lr(5000,X_train,y_train)
linearReg.fit(uniform_X, uniform_y)

pred = linearReg.predict(X_train)
total_cost = np.sum((pred - y_train.reshape(-1)) ** 2) / 463715
print(total_cost)

93.11383677519406


In [286]:
a = 1
for i in range(1000):
    a *= (500000-i) / 500000
print(a)

0.3680022108329327
